In [1]:
import os
import numpy as np
import pandas as pd
import time
import scanpy as sc
import torch

import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=logging.INFO)

import sys
sys.path.append('../../')
import scDualGN
sys.path.append('../')
import expri_helper

/jdfssz1/ST_HEALTH/P18Z10200N0124/AI/user/baiyong/anaconda3/envs/mytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dat_path = expri_helper.get_path('stephenson', dat_ls_file='../expri_data.json')
adata = sc.read_h5ad(dat_path)
adata.X = adata.layers['raw']

In [4]:
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_genes(adata, min_cells=3)


ValueError: cannot specify integer `bins` when input data contains infinity

In [7]:
sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2500)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata,max_value=10)

In [8]:
#kmeans
# start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata, n_cluster=18,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2)

# scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:2', verbosity=True, batch_size=1024).pretrain(alpha=0.05, beta=20, gamma=4)
# scdualgn_model.cluster(n_cluster=18, eta=1, nu=1, n_epochs=30)

lr, alpha, gamma, nu = 0.0001, 0.01, 4, 0.01
print('=====================================')
print('lr:{}, alpha:{}, gamma:{}, nu:{}'.format(lr, alpha, gamma, nu))
start = time.time()
torch.cuda.empty_cache()
scdualgn_model = scDualGN.scDualGN(adata, n_z=32,device='cuda:1', batch_size=1024*2, verbosity=True, lr=lr).pretrain(alpha=alpha, beta=1, gamma=gamma)
scdualgn_model.cluster(n_cluster=18, eta=1, nu=nu, n_epochs=32)


end = time.time()

print('running time = {}'.format(end-start))

INFO:scDualGN.run:dual-VAE pretrain start...


lr:0.0001, alpha:0.01, gamma:4, nu:0.01


INFO:scDualGN.run:Epoch 1/50,Overall loss:2857.1203,MSE:1515.1759,MSE1:335.3044,KL: 72.6684
INFO:scDualGN.run:Epoch 2/50,Overall loss:1017.7283,MSE:920.2604,MSE1:24.0732,KL: 117.4942
INFO:scDualGN.run:Epoch 3/50,Overall loss:931.8932,MSE:887.5150,MSE1:10.7491,KL: 138.1739
INFO:scDualGN.run:Epoch 4/50,Overall loss:903.2705,MSE:874.5350,MSE1:6.8064,KL: 151.0123
INFO:scDualGN.run:Epoch 5/50,Overall loss:887.9120,MSE:865.9474,MSE1:5.0925,KL: 159.4603
INFO:scDualGN.run:Epoch 6/50,Overall loss:878.0410,MSE:859.4701,MSE1:4.2304,KL: 164.9423
INFO:scDualGN.run:Epoch 7/50,Overall loss:871.0715,MSE:854.1919,MSE1:3.7990,KL: 168.3539
INFO:scDualGN.run:Epoch 8/50,Overall loss:866.0957,MSE:850.0830,MSE1:3.5772,KL: 170.4083
INFO:scDualGN.run:Epoch 9/50,Overall loss:863.4789,MSE:847.8956,MSE1:3.4671,KL: 171.5080
INFO:scDualGN.run:Epoch 10/50,Overall loss:862.1440,MSE:846.7534,MSE1:3.4177,KL: 171.9979
INFO:scDualGN.run:Epoch 11/50,Overall loss:853.2050,MSE:839.4421,MSE1:2.9993,KL: 176.5712
INFO:scDualGN

acc 0.7166, nmi 0.6901, ari 0.5803, hs 0.6851, cs 0.6951, purity: 0.7879
running time = 2115.9087710380554


In [ ]:
#leiden
# start = time.time()
# z_final, y_pred_lab, model = scDualGN.run_scDualGN(adata,n_z=32,n_epoch_update_pq=5, n_epochs=30,alpha_dualvae=0.02,beta_daulvae=5,gamma_dualvae=4,GPU_id=2,n_neighbors=50)
# end = time.time()

# print('running time = {}'.format(end-start))